# Homework 1

#### EE-556 Mathematics of Data - Fall 2024

In this homework, we consider a binary classification task that we will model using logistic regression. Your goal will be to find a classifier using first-order methods and accelerated gradient descent methods. The first part will consist of more theoretical questions, and the second one will ask you to implement these methods.

<div class="alert alert-info">
  ℹ️ <strong>Information on group based work:</strong>
</div>

- You are to deliver only 1 notebook per group.
- Asking assistance beyond your group is ok, but answers should be individual to the group.
- In the event that there was <span style="color: red;">disproportional work done</span> by different group members, let the TAs know.

<div style="border: 1px solid #f00; background-color: #fdd; padding: 10px; border-radius: 5px;">
  ⚠️ Do not forget: Write who are the people in your group as well as their respective SCIPER numbers
</div>

Person 1 **Name**: || Person 1 **SCIPER**:


Person 2 **Name**: || Person 2 **SCIPER**:


Person 3 **Name**: || Person 3 **SCIPER**:

## 1. Logistic Regression - 15 Points

Logistic regression is a classic approach to _binary classification_. Before we dive in, let us first define the standard logistic function $\sigma$ on which most of what follows is built:

\begin{equation*}
    \sigma : x \mapsto \frac{1}{1 + \exp{(-x)}}.
\end{equation*}

In logistic regression, we model the _conditional probability_ of observing a class label $b$ given a set of features $\mathbf{a}$. More formally, if we observe $n$ independent samples

\begin{equation*}
    \{(\mathbf{a}_i,b_i)\}_{i=1}^n,
\end{equation*}

where $\mathbf{a}_i\in\mathbb{R}^p$ and $b_i\in\{-1, +1\}$ is the class label, we _assume_ that $b_i$ given $\mathbf{a}_i$ is a symmetric Bernouilli random variable with parameter $\sigma(\mathbf{a}_i^T\mathbf{x}^\natural)$, for some unknown $\mathbf{x}^\natural \in \mathbb{R}^p$. In other words, we assume that there exists an $\mathbf{x}^\natural \in \mathbb{R}^p$ such that

\begin{equation*}
    \mathbb{P}(b_i = 1 \mid \mathbf{a}_i) = \sigma(\mathbf{a}_i^T\mathbf{x}^\natural) \quad \text{ and } \quad \mathbb{P}(b_i = -1 \mid \mathbf{a}_i) = 1 - \sigma(\mathbf{a}_i^T\mathbf{x}^\natural)=  \sigma( - \mathbf{a}_i^T\mathbf{x}^\natural).
\end{equation*}

This is our statistical model. It can be written in a more compact form as follows,

\begin{equation*}
    \mathbb{P}(b_i = j \mid \mathbf{a}_i) = \sigma(j \cdot \mathbf{a}_i^T\mathbf{x}^\natural), \quad j \in \{+1, -1\}.
\end{equation*}

Our goal now is to determine the unknown $\mathbf{x}^\natural$ by constructing an estimator.

__(a)__ (1 point) We are provided with a set of $n$ independent observations. Show that the negative log-likelihood $f$ can be written as:

\begin{equation*}
	\begin{aligned}
		f(\mathbf{x}) = -\log(\mathbb{P}(b_1, \dots, b_n | a_1, \dots, a_n)) & = \sum_{i=1}^n  \log(1 + \exp(- b_i \mathbf{a}_i^T\mathbf{x})).
	\end{aligned}
\end{equation*}


**Solution:**

By definition of loglikelihood:

\begin{equation*}
\log \mathbb{P}(b_1, \dots, b_n \mid \mathbf{a}_1, \dots, \mathbf{a}_n; \mathbf{x}) = \sum_{i=1}^n \log \sigma(b_i \mathbf{a}_i^T \mathbf{x}).
\end{equation*}

Using the definition of $ \sigma(x) = \frac{1}{1 + e^{-x}} $, we can express $ \log \sigma(x) $ as:

\begin{equation*}
\log \sigma(x) = -\log(1 + e^{-x}).
\end{equation*}

Therefore, the negative log-likelihood becomes:

\begin{equation*}
f(\mathbf{x}) = -\sum_{i=1}^n \log \sigma(b_i \mathbf{a}_i^T \mathbf{x}) = \sum_{i=1}^n \log(1 + e^{-b_i \mathbf{a}_i^T \mathbf{x}}).
\end{equation*}
q.e.d.

__(b)__ (2 point) Show that the function $u \mapsto \log(1 + \exp(-u))$ is convex. Deduce that $f$ is convex.

**Solution:**

It is enough to show that $g(x)=\log(1+\exp(-x))$ is twice-diff. and it's second derivative is non-negative for all ral values x.

By the chain rule we have:

\begin{equation*}
g'(x)=\frac{1}{1+\exp(-x)}\cdot(-\exp(-x))=-\frac{1}{\exp(x)+1}
\end{equation*}

\begin{equation*}
g''(x)=\frac{\exp(x)}{(\exp(x)+1)^2} > 0,
\end{equation*}
$\forall x\in \mathbb{R}$. Thus, the second derivative is positive and $g(x)$ is convex. Obv. the sum of convex functions is convex q.e.d.

You have just established that the negative log-likelihood is a convex function. So in principle, any local minimum of the maximum likelihood estimator, which is defined as

\begin{equation*}
    \mathbf{x}^\star_{ML} = \arg\min_{\mathbf{x} \in \mathbb{R}^p} f(\mathbf{x}),
\end{equation*}

is a global minimum so it can serve as an estimator of $\mathbf{x}^\natural$. But, does the minimum always exist? We will ponder this question in the following three points.

__(c)__ (1 point) Explain the difference between infima and minima.  Give an example of a convex function, defined over $\mathbb{R}$, that does not attain its infimum. 

**Solution:**

In general, minimum is an infimum which can be achieved, i.e. stays in the observed set, whereas infimum may not be in the set.

Consider $x \to \exp(x)$. Clearly, it is convex and defined over all real values. 

Still, it's infimum is $0$. For any (small) $\varepsilon > 0:$

\begin{equation*}
\varepsilon = exp(x_1), \quad \text{so} \quad x_1=\log(\varepsilon),
\end{equation*}

but
0  doesn't equal $exp(x_2)$, for any real $x_2.$ q.e.d.

__(d)__ (1 point) Let us assume that there exists $\mathbf{x}_0 \in \mathbb{R}^p$ such that 

\begin{equation*}
    \forall i\in\{1, \dots, n\}, \quad \quad b_i \mathbf{a}_i^T\mathbf{x}_0 > 0.
\end{equation*}

This is called _complete separation_ in the literature. Can you think of a geometric reason why this name is appropriate? Think of a 2D example where this can happen (i.e $p=2$) and describe why _complete separation_ is an appropriate name.



**Solution:**

Geometrically, the $\mathbf{x}_0$ separates 0s and 1s as a hyperspace in the whole space $\mathbb{R}^{p}$.

This hyperspace is given by the set of $n$ equations $\{\mathbf{a}_i^{T}\mathbf{x}_0=0\}_{i=1}^n$ (which, by the algebra theorems, defines a hyperspace as the points $\mathbf{x}_0$ which satisfy the equations). 

In 2D setting, we may have the line which separates the points of two types (like, red's and green's completely). This way we have the indeed complete separation. q.e.d.

From this, you should see that it is likely that our data satisfies the complete separation assumption. Unfortunately, as you will show in the following question, this can become an obstacle.

__(e)__ (1 point) In a _complete separation_ setting, i.e, there exists $\mathbf{x}_0$ such that 

\begin{equation*}
    \forall i\in\{1, \dots, n\}, \quad \quad b_i \mathbf{a}_i^T\mathbf{x}_0 > 0,
\end{equation*}

prove that the function $f$ does not attain its minimum. 

__Hint__: If the function did have a minimum, would it be above, below or equal to zero? Then think of how $f(2 \mathbf{x}_0)$ compares with $f(\mathbf{x}_0)$, how about $f(\alpha \mathbf{x}_0)$ for $\alpha \rightarrow + \infty$ ? 

**Solution:**

It is easy to see that for $\alpha\to+\infty$:

\begin{equation*}
-b_i \mathbf{a}_i^T(\alpha\mathbf{x}_0) = \alpha\cdot(- b_i \mathbf{a}_i^T\mathbf{x}_0) \to -\infty.
\end{equation*}

This means that 
\begin{equation*}
\exp(-b_i \mathbf{a}_i^T(\alpha\mathbf{x}_0))\to 0+,\text{ as }\alpha\to+\infty.
\end{equation*}
So,
\begin{equation*}
\lim_{\alpha\to+\infty}\log(1 + \exp(- b_i \mathbf{a}_i^T(\alpha\mathbf{x}_0))=
\lim_{\alpha\to+\infty}\log(1+0+)=0+.
\end{equation*}
Here $0+$ means the right convergence to the corresponding value. Thus, the infimum of $f(x)$ is 0 and it is not achieveable, as $f$ is always non-negative. q.e.d. 

As you have just shown convex functions do not always attain their infimum. So it is possible for the maximum-likelihood estimator $\mathbf{x}^\star_{ML}$ to not exist. We will resolve this issue by adding a regularizer. 

In what follows, we consider the function

\begin{equation*}
	f_\mu(\mathbf{x}) = f(\mathbf{x}) + \frac{\mu}{2}\|\mathbf{x}\|_2^2
\end{equation*}
with $\mu> 0$.

__(f)__ (1 point) Show that the gradient of $f_\mu$ can be expressed as 
\begin{equation}
	\nabla f_\mu(\mathbf{x}) = \sum_{i=1}^n -b_i \sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x})\mathbf{a}_i + \mu \mathbf{x}.
\tag{1}
\end{equation}
__Hint__: Lecture 3 shows you how to proceed with this question.

**Solution:**

Compute the derivative of one term

\begin{equation*}
\frac{d}{dx} \log(1 + \exp(-b_i \mathbf{a}_i^T \mathbf{x})) = \frac{1}{1 + \exp(-b_i \mathbf{a}_i^T \mathbf{x})} \cdot \frac{d}{dx} \left(1 + \exp(-b_i \mathbf{a}_i^T \mathbf{x})\right).
\end{equation*}

Now, 

\begin{equation*}
\frac{d}{dx} \exp(-b_i \mathbf{a}_i^T \mathbf{x}) = -b_i \exp(-b_i \mathbf{a}_i^T \mathbf{x}) \cdot \mathbf{a}_i.
\end{equation*}

Thus, the gradient of $ f_i(\mathbf{x}) $ becomes

\begin{equation*}
\nabla f_i(\mathbf{x}) = \frac{-b_i \exp(-b_i \mathbf{a}_i^T \mathbf{x}) \mathbf{a}_i}{1 + \exp(-b_i \mathbf{a}_i^T \mathbf{x})}
= -b_i\sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x})\mathbf{a}_i
\end{equation*}

We also clearly have that 
\begin{equation*}
\frac{d}{dx} \frac{\mu}{2}||\mathbf{x}||_2^2 = \frac{d}{dx} \frac{\mu}{2}\sum_{i=1}^p (x_i^2)  = \frac{\mu}{2}\cdot2\cdot \mathbf{x}=\mu\mathbf{x}.
\end{equation*}

Therefore 
\begin{equation*}
\nabla \frac{\mu}{2}||\mathbf{x}||_2^2 = \mu\mathbf{x}.
\end{equation*}

The resulting gradient of the whole $f_{\mu}(x)$ is just the whole sum of gradients of the terms and the regularizer. q.e.d.

__(g)__ (1 point) Show that the Hessian of $f_\mu$ can be expressed as 
\begin{equation}
	\nabla^2 f_\mu(\mathbf{x}) = \sum_{i=1}^{n} \sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x})(1 - \sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x}))\mathbf{a}_i\mathbf{a}_i^T + \mu \mathbf{I}.
\tag{2}
\end{equation}

**Solution:**

The Hessian of $f_{\mu}$ is the $\nabla$ of the $\nabla f_{\mu}(x)$.

Again, computing for each term we have:

\begin{equation*}
\nabla^2 f_i(\mathbf{x}) 
=\nabla (-b_i\sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x})\mathbf{a}_i)
=-b_i\nabla(\sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x}))\mathbf{a}_i.
\end{equation*}

As we know, $\nabla\sigma(\mathbf{x})=\sigma(\mathbf{x})(1-\sigma(\mathbf{x})).$ Thus, by the chain rule:

\begin{equation*}
\nabla\sigma(-b_i\mathbf{a}_i^T\mathbf{x})=
\sigma(-b_i\mathbf{a}_i^T\mathbf{x})(1-\sigma(-b_i\mathbf{a}_i^T\mathbf{x}))\cdot (-b_i\cdot\mathbf{a}_i^T).
\end{equation*}

Since $b_i=\pm 1$, $b_i^2=1$. So

\begin{equation*}
\nabla^2 f_i(\mathbf{x}) = 
\sigma(-b_i\mathbf{a}_i^T\mathbf{x})(1-\sigma(-b_i\mathbf{a}_i^T\mathbf{x}))\mathbf{a}_i\mathbf{a}_i^T.
\end{equation*}

Clearly, derivative w.r. to $\mathbf{x}$ of $\mu\mathbf{x}$ is just $\mu$, which transforms to $\mu\mathbf{I}$ in the matrix case.

Now, the Hessian of $f_\mu(x)$ is just the sum of $\nabla^2 f_i(x)$ and hessian of regularizer. q.e.d.

It is convenient to observe that we can write the Hessian in a more compact form by defining the matrix
\begin{equation}
	\mathbf{A} = \begin{bmatrix}
        \leftarrow &  \mathbf{a}_1^T & \rightarrow \\
        \leftarrow &  \mathbf{a}_2^T & \rightarrow \\
         &  \ldots &  \\
        \leftarrow &  \mathbf{a}_n^T & \rightarrow \\
  \end{bmatrix}.
\end{equation}
It is easy to see that we have
\begin{equation}
	\nabla^2 f_\mu(\mathbf{x}) =  \mathbf{A}^T \text{Diag}\left(\sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x})(1 - \sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x})) \right)\mathbf{A}+ \mu \mathbf{I}.
\end{equation}


__(h)__ (1 point) Show that $f_\mu$ is $\mu$-strongly convex. 

**Solution:**

We may use the $\mu$-strong convexity def. in terms of Hessian of $f_\mu(x)$, whose minimum eigenvalue must be at least $\mu$.

Following the **(g)** we have that the matrix $\mathbf{A}^T \text{Diag}\left(\sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x})(1 - \sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x})) \right)\mathbf{A}$ should have eigenvalues at least 0.

Obviously, $\text{Diag}\left(\sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x})(1 - \sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x})) \right)$ does not affect the positiveness of the Hessian (values of $\sigma$ are positive and less than $1$ anyways). 

So in the end we need to know if $A^TA$ has non-negative eigenvalues, which is true:

$$
x^TA^TAx= (Ax)^TAx = ||Ax||^2_2 \geq 0,
$$
for every non-zero vector $x$. Thus, this matrix is positive semi-definite, or have non-negative eigenvalues, and the proof is complete. q.e.d.

__(i)__ (3 points) Is it possible for a strongly convex function to not attain its minimum ? <a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1) Justify your reasoning.

**Solution:**

We know that for a $\mu$-strongly convex func. $f(x)$ there exists convex $h(x)$ (with the same domain), such that
$$
h(x) = f(x)-\frac{\mu}{2}||x||_2^2.
$$
Let's fix $\mathbf{x}_0$ from the domain of $f$. As $h$ is convex, then it has nonempty subgradient set $\partial h$ for each $\mathbf{x}_0$, say $w\in\partial h(\mathbf{x}_0)$,
such that
$$
w\cdot (\mathbf{x}-\mathbf{x}_0)\leq h(\mathbf{x})-h(\mathbf{x}_0),
$$ where $\cdot$ means dot-product of two vectors. Thus
$$
w\cdot (\mathbf{x}-\mathbf{x}_0)+\frac{\mu}{2}||x||_2^2\leq f(\mathbf{x})-h(\mathbf{x}_0).
$$
From the last inequality we have that $f(\mathbf{x})\to+\infty$ as $x\to\infty$ (because of the $||x||_2^2$, which obv. goes to $+\infty$ faster than the dot-product potentially can).

This means that the level-set 
$$
A(\mathbf{x}_0)=\{x|f(x)\leq f(x_0) \}
$$ is bounded (in other case, we select sequence such that $f(x)\to+\infty$).

Now, if 
$$
\{\mathbf{x}_k\}\subset A(\mathbf{x}_0):\lim_{k\to\infty}\mathbf{x}_k=\mathbf{x},
$$
then we have
$$
f(\mathbf{x})\leq\liminf_{k\to\infty}f(\mathbf{x}_k)\leq f(\mathbf{x}_0),
$$
where the first $\leq$ is true because $f$ is convex (thus, lower-semicontinuous), and the second $\leq$ is true because each $f(\mathbf{x}_k)\leq f(\mathbf{x}_0)$ (being from the level-set). 

This means that $A$ is closed and bounded, so compact. Thus, by continuity $f$ attains it's minimum on $A$, and, by the def. of $A$, minimizer on A is also a global minimizer. q.e.d.

We will now show that $f_\mu$ is smooth, i.e, $\nabla f_\mu$ is L-Lipschitz with respect to the Euclidean norm, with 
\begin{equation}
	L = \|A\|^2_F + \mu \text{, where }\|\cdot\|_F\text{ denotes the Frobenius norm. }
\end{equation}

(1 point for all three questions)

__(j-1)__ Show that $\lambda_{\max}(\mathbf{a}_i\mathbf{a}_i^T) = \left\| \mathbf{a}_i\right\|_2^2$, where $\lambda_{\max}(\cdot)$ denotes the largest eigenvalue.

**Solution:**

We know that matrix $rank(\mathbf{a}_i\mathbf{a}_i^T)=1$, as it is an outer product of two vectors (so the columns are all prop. to a_i), thus it has only one non-zero eigenvalue. Now
$$
\mathbf{a}_i\mathbf{a}_i^T\mathbf{a}_i = \mathbf{a}_i\cdot(\mathbf{a}_i^T\mathbf{a}_i) = ||\mathbf{a}_i||_2^2 \mathbf{a}_i,
$$
thus $||\mathbf{a}_i||_2^2$ is indeed an eigenvalue. As it is unique non-zero, it is then $\lambda_{\max}$. q.e.d.

__(j-2)__ Using [2](#mjx-eqn-eq2), show that $\lambda_{\max}(\nabla^2 f_\mu(\mathbf{x})) \leq \sum_{i=1}^{n} \|\mathbf{a}_i\|_2^2 + \mu$. 

__Hint__: Recall that $\lambda_{\max}(\cdot)$ verifies the triangle inequality.

**Solution:**

We use the triangle inequality for $\lambda_\max$. So
$$
\lambda_{\max}(\nabla^2 f_\mu(\mathbf{x})) \leq \lambda_\max(\mathbf{A}^T \text{Diag}\left(\sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x})(1 - \sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x})) \right)\mathbf{A}) + \lambda_\max(\mu \mathbf{I}).
$$
We obv. have 
$$
\lambda_\max(\mu \mathbf{I}) = \mu,
$$
And each value in $\text{Diag}\left(\sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x})(1 - \sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x})) \right)$ can't be more than 1 and less than 0, so 
$$
\lambda_\max(\mathbf{A}^T \text{Diag}\left(\sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x})(1 - \sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x})) \right)\mathbf{A})\leq \lambda_\max(\mathbf{A}^T\mathbf{A}).
$$
Now, for the Frobenius norm of $\mathbf{A}$ we have 
$$
\sigma_{max}^2\leq\sum_{i=1}^n\sigma_i^2 = ||\mathbf{A}||^2_F = \sum_{i=1}^n||\mathbf{a}_i||_2^2,
$$
where $\sigma_i$ are the singular values of $\mathbf{A}$, and $\sigma_{max}^2=\lambda_{\max}(\mathbf{A}^T\mathbf{A}).$ q.e.d.

__(j-3)__ Conclude that $f_\mu$ is $L$-smooth for $L = \|A\|_F^2 + \mu$.

**Solution:**

We just see that the largest eigenvalue of the Hessian is upper-bounded by $\|\mathbf{A}\|_F^2 + \mu$, and that is exavtly the def. of L-smoothness for the desired L. q.e.d.

__(l)__ (2 point) To finalize, we introduce the Kullback-Leibler (KL) divergence. The KL divergence is a measure of how one probability distribution differs from a second, reference probability distribution. Along side the examples we saw in slide 18 of Lecture 1, the KL divergence is also a useful loss function to be used in learning frameworks.

Write the definition of the Kullback-Leibler (KL) divergence between the true label distribution $q(b_i)$ and the model’s predicted distribution $p(b_i∣\mathbf{a}_i)$ and show that minimizing the KL divergence between $q(b_i)$ and $p(b_i∣\mathbf{a}_i)$ is equivalent to minimizing the negative log-likelihood derived in (a).

**Solution:**

We have the def. of DKL:
$$
\mathbf{D}_{KL}(q(b)||p(b|\mathbf{a})) = \mathbb{E}_{b\sim q(b|\mathbf{a})}\log\left(\frac{q(b|\mathbf{a})}{p(b|\mathbf{a})}\right)
=\mathbb{E}_{b\sim q(b|\mathbf{a})}(\log(q(b|\mathbf{a})))-\mathbb{E}_{b\sim q(b|\mathbf{a})}(\log(p(b|\mathbf{a}))),
$$
and as the first term does not matter within the argmin problem, we have
$$
\textbf{argmin}_{b}\mathbf{D}_{KL}(q(b)||p(b|\mathbf{a})) = \textbf{argmin}_{b}(-\mathbb{E}_{b\sim q(b|\mathbf{a})}(\log(p(b|\mathbf{a})))_,
$$
and then obv. we can write
$$
-\mathbb{E}_{b\sim q(b|\mathbf{a})}(\log(p(b|\mathbf{a})))=-\frac{1}{n}\sum_{i=1}^n \log p(b_i|\mathbf{a}_i),
$$
which completes the proof. q.e.d.



From your work in this section, you have shown that the maximum likelihood estimator for logistic regression might not exist, but it can be guaranteed to exist by adding a $\|\cdot\|_2^2$ regularizer. Consequently, the estimator for $\mathbf{x}^\natural$ we will use will be the solution of the smooth strongly convex problem,
\begin{equation}
	\mathbf{x}^\star=\arg\min_{\mathbf{x} \in \mathbb{R}^p} f(\mathbf{x}) + \frac{\mu}{2}\|\mathbf{x}\|_2^2.
\tag{3}
\end{equation}

<a name="cite_note-1"></a>1. [^](#cite_ref-1) TA's will give you candy if you provide a complete proof.